In [1]:
import numpy as np 
from policy_gradient import PolicyGradient
import matplotlib.pyplot as plt

In [2]:
import random
import numpy as np
from statistics import median, mean
from collections import Counter

In [3]:
# import game env
from puzzle import GameGrid
env = GameGrid()
env.reset()

In [4]:
RENDER_ENV = True
EPISODES = 500
rewards = []
RENDER_REWARD_MIN = 50
LR = 0.01

In [5]:
load_path = None #"output/weights/CartPole-v0.ckpt"
save_path = "output/weights/experiment2.ckpt"

PG = PolicyGradient(
    n_x = 16,
    n_y = 4,
    learning_rate=LR,
    reward_decay=0.95,
    load_path=load_path,
    save_path=save_path
)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
for episode in range(EPISODES):
    print("Episode: ", episode)
    observation = env.reset()
    episode_reward = 0
    
    while True:
        if RENDER_ENV: env.render()

In [ ]:
# Test the policy network
num_games = 1000
goal_steps = 200
rewards = []
choices = []
accepted_scores = []

for each_game in range(num_games):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        #env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,4)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0]).item()
                
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        new_observation = np.array(np.array(new_observation).flatten().tolist())
        
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        
        score+=reward
        if done: break
        
    if each_game%100 == 0: print(each_game)
    rewards.append(score)
    accepted_scores.append(env.highest_score())

print('Average Score:',sum(rewards)/len(rewards))
print('choice 0:{} choice 1:{} choice 2:{} choice 3:{}'.format(choices.count(0)/len(choices),choices.count(1)/len(choices), choices.count(2)/len(choices), choices.count(3)/len(choices)))
print(Counter(accepted_scores))

0
100
200
300
400
500
600
700
